In [1]:
import numpy as np
import copy

In [2]:
der = {-1:'-', 0:'0', 1:'+'}
mag = {0:'0', 1:'+', 2:'max'}

class Inflow:
    def __init__(self, magnitude, derivative):
            if magnitude in [0, 1] and derivative in [-1, 0, 1]: #and not((derivative == -1) and magnitude == 0):
                self.magnitude = magnitude
                self.derivative = derivative
            else:
                print('Wrong arguments')
                
                
    def upd_inflow(self, new_derivative):
            if (new_derivative == -1 and self.magnitude == 0) or new_derivative not in [-1, 0, 1]:
                    print('Impossible state')
                    return self
            else:
                self.derivative = new_derivative
                return self
            
    def print_state(self):
            print('Inflow: M = '+str(mag[self.magnitude] +' D = '+ der[self.derivative]))
            
    def compare(self, i1):
        if self.magnitude == i1.magnitude and self.derivative == i1.derivative:
            return True
        else:
            return False
        
    def is_possible(self):
        if self.magnitude == 0 and self.derivative == -1: 
            return False
        else:
            return True
    
        
class Volume:
    def __init__(self, magnitude, derivative):
            
            if magnitude in [0, 1, 2] and derivative in [-1, 0, 1]:
                self.magnitude = magnitude
                self.derivative = derivative
            else:
                print('Wrong arguments')
                
    def print_state(self):
            print('Volume: M = '+str(mag[self.magnitude] +' D = '+ der[self.derivative]))
            
    def compare(self, i1):
        if self.magnitude == i1.magnitude and self.derivative == i1.derivative:
            return True
        else:
            return False
        
    def is_possible(self):
        if (self.magnitude == 2 and self.derivative == 1) or (self.magnitude == 0 and self.derivative == -1):
            return False
        else:
            return True
    
                

        
class Outflow:
    def __init__(self, magnitude, derivative):
            
            if magnitude in [0, 1, 2] and derivative in [-1, 0, 1]:
                self.magnitude = magnitude
                self.derivative = derivative
            else:
                print('Wrong arguments')
                
    def print_state(self):
            print('Outflow: M = '+str(mag[self.magnitude] +' D = '+ der[self.derivative]))
            
    def compare(self, i1):
        if self.magnitude == i1.magnitude and self.derivative == i1.derivative:
            return True
        else:
            return False
        
        
class State:
    def __init__(self, id1, inflow, volume, outflow):
        if volume.derivative == outflow.derivative and volume.magnitude == outflow.magnitude:
            self.id = id1
            self.inflow = inflow
            self.volume = volume
            self.outflow = outflow
            #self.previous = []
            #self.next = []
            self.connection = []
        else:
            print('Impossible state')
            
    def edge_title(self):
        text = str('I: '+mag[self.inflow.magnitude] +' '+ der[self.inflow.derivative]+'\n'+ 'V: '+mag[self.volume.magnitude] +' '+ der[self.volume.derivative] + '\n' + 'O: '+mag[self.outflow.magnitude] +' '+ der[self.outflow.derivative] + '\n') 
        return text
    
    def add_connection(self, id_connection):
        if id_connection not in self.connection:
            self.connection.append(id_connection)
        
    def print_state(self):
        print('ID ', self.id)
        self.inflow.print_state()
        self.volume.print_state()
        self.outflow.print_state()
        print('Connection', self.connection)
        
    def compare(self, s1):
        if self.inflow.magnitude == s1.inflow.magnitude and self.inflow.derivative == s1.inflow.derivative and self.outflow.magnitude == s1.outflow.magnitude and self.outflow.derivative == s1.outflow.derivative and self.volume.magnitude == s1.volume.magnitude and self.volume.derivative == s1.volume.derivative:
            return True
        else:
            return False
            
    def is_possible(self):
        if self.inflow.magnitude == 1 and self.volume.magnitude == 0 and self.volume.derivative == 0:
            return False
        else:
            return True

In [3]:
def get_possible_derivatives_inflow(inflow):
    if inflow.magnitude == 0 and inflow.derivative == -1:
        return [0]
    elif inflow.derivative == -1:
        return [-1, 0]
    elif inflow.magnitude == 0 and inflow.derivative == 0:
        return [1, 0]
    elif inflow.magnitude != 0 and inflow.derivative == 0:
        return [-1, 0, 1]
    elif inflow.derivative == 1:
        return [1, 0]


def get_new_states_inflow(inflow):
    '''
    return: list of possile inflow states
    (return dict with flag succescfl or not an list)
    '''
    new_derivative = inflow.derivative
    if new_derivative == -1 and inflow.magnitude == 0:
        print('impossible change')
        return {'flag':-1,'result':[inflow]}
    elif new_derivative == 1 and inflow.magnitude == 0:
        return {'flag':1,'result':[Inflow(1 , 1)]}
    
    elif new_derivative == 1 and inflow.magnitude == 1:
        return {'flag':1,'result':[Inflow(1 , 1)]}
    
    elif new_derivative == 0 and inflow.magnitude == 0:
        return {'flag':1,'result':[Inflow(0 , 0)]}
    
    elif new_derivative == 0 and inflow.magnitude == 1:
        return {'flag':1,'result':[Inflow(1, 0)]}
    
    elif new_derivative == -1 and inflow.magnitude == 1:
        return {'flag':2,'result':[Inflow(1 ,-1), Inflow(0 , -1)]}


def get_all_states_inflow(inflow):
    '''
    return: list of possile inflow states given updated derivative
    (return dict with flag succescfl or not an list)
    '''
    results = []
    results_ = []
    derivatives = get_possible_derivatives_inflow(inflow)

    for new_derivative in derivatives:
        if Inflow(inflow.magnitude, new_derivative).is_possible():
            results.append(Inflow(inflow.magnitude, new_derivative))
        
    res = get_new_states_inflow(inflow)
    if res['flag'] != -1:
            for i in res['result']:
                if i.is_possible():
                    results.append(i)
                
    return results



In [4]:
def get_possible_derivatives_volume(volume, outflow, inflow):
    '''
    '''
    if inflow.magnitude == 0 and outflow.magnitude == 0:
        return [0]
    elif inflow.magnitude == 0 and outflow.magnitude != 0:
        return [-1]
    elif inflow.magnitude == 1 and outflow.magnitude == 0:
        return [1]
    else:
        if volume.derivative == -1:
            return [-1, 0]
        elif volume.derivative == 0:
            return [1, 0, -1]
        elif volume.derivative == 1:
            return [1, 0]

def get_new_states_volume(volume):
    '''
    given the volume get possible changes in it .
    
    input: volume - the old state of volume, new_derivative.
    return: dict with flag: -1 if the update is not succesful, otherwise identifies the length of result. 
             result - possible state of volumes.
    '''
    
    new_derivative = volume.derivative
    if (new_derivative == -1 and volume.magnitude == 0) or (new_derivative == 1 and volume.magnitude == 2):
        return {'flag':-1,'result':[volume]}
    
    elif (new_derivative == 1 and volume.magnitude == 0) or (new_derivative == -1 and volume.magnitude == 1):
        return {'flag':2,'result':[Volume(0, new_derivative), Volume(1, new_derivative)]}
    
    elif (new_derivative == 1 and volume.magnitude == 1):
        return {'flag':2,'result':[Volume(2, 0), Volume(1, new_derivative)]}
    
    elif (new_derivative == -1 and volume.magnitude == 2):
        return {'flag':2,'result':[Volume(2, new_derivative), Volume(1, new_derivative)]}
    
    elif new_derivative == 0:
        return {'flag':1,'result':[Volume(volume.magnitude, new_derivative)]}
    
    
def get_all_states_volume(volume, outflow, inflow):
    '''
    return: list of possile inflow states given updated derivative
    (return dict with flag succescfl or not an list)
    '''
    results = []

    derivatives = get_possible_derivatives_volume(volume, outflow, inflow)

    for new_derivative in derivatives:
        if Volume(volume.magnitude, new_derivative).is_possible():
            results.append(Volume(volume.magnitude, new_derivative))
        
    res = get_new_states_volume(volume)
    if res['flag'] != -1:
            for i in res['result']:
                if i.is_possible():
                    results.append(i)
                

    return results




In [5]:
def get_possible_states(old_state, states_id, all_states):
    results = []
    #res1 = get_new_states_inflow(old_state.inflow, new_derivative_inflow)
    #print(res1)
    #if res1['flag'] == -1:
    #    print('impossible operation')
    #    return old_state
    
    res1 = get_all_states_inflow(old_state.inflow)
    for new_inflow in res1:
        res2 = get_all_states_volume(old_state.volume,old_state.outflow,new_inflow)
        for new_volume in res2:
                        
                        # important compare with another!!! states
                        
                    i = copy.deepcopy(new_inflow)
                    v = copy.deepcopy(new_volume)
                    o = Outflow(new_volume.magnitude, new_volume.derivative)
                    st_id = max(states_id) + 1
                    s = State(st_id, i, v, o)
                    
                    if s.is_possible():
                        if not(s.compare(old_state)):
                            k=False
                            
                            for s1 in results:
                                if s.compare(s1):
                                    k = True
                                    
                            for s1 in all_states:
                                if s.compare(s1):
                                    k = True
                                    old_state.add_connection(s1.id)
                                    
                            if k == False:
                                states_id.append(st_id)
                                results.append(s)
                                all_states.append(s)
                                old_state.add_connection(s.id)
                                
                        
    return results, all_states

In [6]:
i = Inflow(0, 1)
o = Outflow(0, 0)
v = Volume(0, 0)

states_id = [0]

starting = State(0, i, v, o)

all_states = [starting]

for s in all_states:
    print(s.id)
    r, a_s = get_possible_states(s, states_id, all_states)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [7]:
for s in all_states:
    s.print_state()

ID  0
Inflow: M = 0 D = +
Volume: M = 0 D = 0
Outflow: M = 0 D = 0
Connection [1, 2]
ID  1
Inflow: M = 0 D = 0
Volume: M = 0 D = 0
Outflow: M = 0 D = 0
Connection [0]
ID  2
Inflow: M = + D = +
Volume: M = 0 D = +
Outflow: M = 0 D = +
Connection [3, 4, 5]
ID  3
Inflow: M = + D = +
Volume: M = + D = +
Outflow: M = + D = +
Connection [6, 7, 5, 8, 9]
ID  4
Inflow: M = + D = 0
Volume: M = 0 D = +
Outflow: M = 0 D = +
Connection [10, 11, 5, 2, 3]
ID  5
Inflow: M = + D = 0
Volume: M = + D = +
Outflow: M = + D = +
Connection [11, 12, 13, 8, 9, 3, 6, 7]
ID  6
Inflow: M = + D = +
Volume: M = + D = 0
Outflow: M = + D = 0
Connection [3, 14, 5, 8, 15]
ID  7
Inflow: M = + D = +
Volume: M = max D = 0
Outflow: M = max D = 0
Connection [16, 9, 17]
ID  8
Inflow: M = + D = 0
Volume: M = + D = 0
Outflow: M = + D = 0
Connection [11, 12, 18, 5, 15, 3, 6, 14]
ID  9
Inflow: M = + D = 0
Volume: M = max D = 0
Outflow: M = max D = 0
Connection [13, 19, 17, 7, 16]
ID  10
Inflow: M = + D = -
Volume: M = 0 D = +
Ou

In [21]:
def get_valid_states(all_states, list_of_states):
    valid_states = copy.deepcopy(np.array(all_states)[list_of_states])
    for i in range(len(valid_states)):
        to = list(valid_states[i].connection_to)
        from_ = list(valid_states[i].connection_from)
        for j in to:
            if j not in list_of_states:
                valid_states[i].connection_to.remove(j)
        for k in from_:
            if k not in list_of_states:
                valid_states[i].connection_from.remove(k)

    trans = {}
    for i in range(len(list_of_states)):
        trans[list_of_states[i]] = i

    for i in range(len(valid_states)):
        to = list(valid_states[i].connection_to)
        from_ = list(valid_states[i].connection_from)
        new_to = []
        new_from = []
        for j in to:
            try:
                new_to.append(trans[j])
            except KeyError:
                new_to.append(j)
        for k in from_:
            try:
                new_from.append(trans[k])
            except KeyError:
                new_from.append(k)
        valid_states[i].connection_to = set(new_to)
        valid_states[i].connection_from = set(new_from)
    return valid_states

valid_states = all_states
self_loop_states = [1, 3, 5, 7, 8, 9, 11, 12, 13, 15, 18, 19]
for state in self_loop_states:
    valid_states[state].connection.append(state)

In [22]:
import pygraphviz as pgv

graph = pgv.AGraph(directed=True, fixedsize=True)

def graph_state(state):
    
    def tm(val): #translate magnitude
        if val == 1:
            return '+'
        elif val == 2:
            return 'max'
        else:
            return val
    def td(val): #translate derivative
        if val == 1:
            return '+'
        elif val == -1:
            return '-'
        else:
            return val
        
    return 'I(%s, %s)\nV(%s, %s)\nO(%s, %s)' % (tm(state.inflow.magnitude), \
                                               td(state.inflow.derivative), \
                                               tm(state.volume.magnitude), \
                                               td(state.volume.derivative), \
                                               tm(state.outflow.magnitude), \
                                               td(state.outflow.derivative)) 

for i in range(len(valid_states)):
    graph.add_node(graph_state(valid_states[i]), color='red', style = 'filled, bold')
num_edges = 0
for i in range(len(valid_states)):
    for j in valid_states[i].connection:
        num_edges+=1
        graph.add_edge(graph_state(valid_states[i]), graph_state(valid_states[j]), timestep = True)

print('#edges ', num_edges)        
graph.layout(prog='dot')

graph.draw('graph1_expected.png')

#edges  102


In [23]:
for state in valid_states:
    state.print_state()

ID  0
Inflow: M = 0 D = +
Volume: M = 0 D = 0
Outflow: M = 0 D = 0
Connection [1, 2]
ID  1
Inflow: M = 0 D = 0
Volume: M = 0 D = 0
Outflow: M = 0 D = 0
Connection [0, 1]
ID  2
Inflow: M = + D = +
Volume: M = 0 D = +
Outflow: M = 0 D = +
Connection [3, 4, 5]
ID  3
Inflow: M = + D = +
Volume: M = + D = +
Outflow: M = + D = +
Connection [6, 7, 5, 8, 9, 3]
ID  4
Inflow: M = + D = 0
Volume: M = 0 D = +
Outflow: M = 0 D = +
Connection [10, 11, 5, 2, 3]
ID  5
Inflow: M = + D = 0
Volume: M = + D = +
Outflow: M = + D = +
Connection [11, 12, 13, 8, 9, 3, 6, 7, 5]
ID  6
Inflow: M = + D = +
Volume: M = + D = 0
Outflow: M = + D = 0
Connection [3, 14, 5, 8, 15]
ID  7
Inflow: M = + D = +
Volume: M = max D = 0
Outflow: M = max D = 0
Connection [16, 9, 17, 7]
ID  8
Inflow: M = + D = 0
Volume: M = + D = 0
Outflow: M = + D = 0
Connection [11, 12, 18, 5, 15, 3, 6, 14, 8]
ID  9
Inflow: M = + D = 0
Volume: M = max D = 0
Outflow: M = max D = 0
Connection [13, 19, 17, 7, 16, 9]
ID  10
Inflow: M = + D = -
Volu

In [24]:
len(valid_states)

20